In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


# `Load datasets`

In [ ]:
X_train = np.load('/kaggle/input/pit-classification/distance_vectors_train.npy')
X_test = np.load('/kaggle/input/pit-classification/distance_vectors_test.npy')
X_val = np.load('/kaggle/input/pit-classification/distance_vectors_val.npy')
y_train = np.load('/kaggle/input/pit-classification/train_labels.npy')
y_test = np.load('/kaggle/input/pit-classification/test_labels.npy')
y_val = np.load('/kaggle/input/pit-classification/val_labels.npy')



# `Convert one-hot encoded labels to class indices`

In [ ]:
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)
y_val_labels = np.argmax(y_val, axis=1)

# `Standardize features`

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_val_scaled = scaler.transform(X_val)

# `Define MLP model`

In [ ]:
model = Sequential([
    Dense(1024, activation='relu', input_shape=(X_train.shape[1],)),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    
    Dense(256, activation='relu'),
    Dropout(0.3),
    
    Dense(y_train.shape[1], activation='softmax')
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5)

# `Train the model`

In [ ]:
history = model.fit(X_train_scaled, y_train, 
                    epochs=100, batch_size=32,
                    validation_data=(X_val_scaled, y_val),
                    callbacks=[early_stopping, lr_scheduler],
                    verbose=1)

Epoch 1/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 72s 69ms/step - accuracy: 0.0062 - loss: 6.3290 - val_accuracy: 0.0217 - val_loss: 5.4266 - learning_rate: 0.0010
Epoch 2/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 63s 63ms/step - accuracy: 0.0188 - loss: 5.4471 - val_accuracy: 0.0422 - val_loss: 5.0434 - learning_rate: 0.0010
Epoch 3/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 66s 66ms/step - accuracy: 0.0320 - loss: 5.1454 - val_accuracy: 0.0642 - val_loss: 4.7933 - learning_rate: 0.0010
Epoch 4/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 57s 57ms/step - accuracy: 0.0484 - loss: 4.9372 - val_accuracy: 0.0765 - val_loss: 4.6545 - learning_rate: 0.0010
Epoch 5/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 61s 62ms/step - accuracy: 0.0587 - loss: 4.7639 - val_accuracy: 0.0873 - val_loss: 4.5717 - learning_rate: 0.0010
Epoch 6/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 79s 58ms/step - accuracy: 0.0697 - loss: 4.6316 - val_accuracy: 0.0958 - val_loss: 4.4803 - learning_rate: 0.0010
Epoch 7/100
998/998 ━━━━━━━━━━━━━━━━━━━━ 62s 62ms/step - accuracy: 0.0

# `Save model and scaler`

In [ ]:

model.save("/kaggle/working/mlp_optimized.h5")
joblib.dump(scaler, "/kaggle/working/scaler.pkl")

['/kaggle/working/scaler.pkl']

# `Testing performance`

In [9]:
# Predictions for each dataset
y_train_pred = np.argmax(model.predict(X_train_scaled), axis=1)
y_val_pred = np.argmax(model.predict(X_val_scaled), axis=1)
y_test_pred = np.argmax(model.predict(X_test_scaled), axis=1)

998/998 ━━━━━━━━━━━━━━━━━━━━ 12s 12ms/step
133/133 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step


In [10]:
# Compute accuracy for train, validation, and test sets
train_accuracy = accuracy_score(y_train_labels, y_train_pred)
val_accuracy = accuracy_score(y_val_labels, y_val_pred)
test_accuracy = accuracy_score(y_test_labels, y_test_pred)

In [11]:
# Print accuracies
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Training Accuracy: 69.52%
Validation Accuracy: 33.20%
Test Accuracy: 34.08%
